In [5]:
from ultralytics import YOLO
import cv2
import numpy as np
from collections import defaultdict
from cap_from_youtube import cap_from_youtube
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # to avoid OMP conficts

In [6]:
# Load an official or custom model
model = YOLO('../models/yolov8n.pt')  # Load an official Detect model

In [9]:
# Open the video file
# fname="highway_drone_footage"
# fname="apollo15_10_1_1st"
# fname="cars"
# cap = cv2.VideoCapture("../videos/"+fname+".mp4")

# Open the video from YouTube
fname="from_yt_5"
# cap = cap_from_youtube("https://youtu.be/7HaJArMDKgI?si=T0Bb3zOcz-YiOnMF", '720p') # 1
# cap = cap_from_youtube("https://youtu.be/Y1jTEyb3wiI?si=-gU51avblW5Qo-ij", '720p') # 2
# cap = cap_from_youtube("https://youtu.be/jM2VrPE5kFg?si=RXfsB63fA58TuBSi", '720p') # 3
# cap = cap_from_youtube("https://youtu.be/CftLBPI1Ga4?si=N69rX1Bo3w9S6qjq", '1080p') # 4 (busy street with cars)
cap = cap_from_youtube("https://youtu.be/dDE3d6mEfn8?si=s5fAKjGVaHhfeLCQ", '2160p50') # 4 (mumbai street)


[youtube] Extracting URL: https://youtu.be/dDE3d6mEfn8?si=s5fAKjGVaHhfeLCQ
[youtube] dDE3d6mEfn8: Downloading webpage
[youtube] dDE3d6mEfn8: Downloading ios player API JSON
[youtube] dDE3d6mEfn8: Downloading android player API JSON
[youtube] dDE3d6mEfn8: Downloading player e04b3b4c
[youtube] dDE3d6mEfn8: Downloading m3u8 information


In [10]:
# Store the track history
track_history = defaultdict(lambda: [])

# Below VideoWriter object will create a frame of above defined The output  
# is stored in 'output.mp4' file. 
output_video = cv2.VideoWriter("../output/"+fname+"_output.mp4",  
                         cv2.VideoWriter_fourcc(*'mp4v'), # writer object # XVID
                        #  cv2.VideoWriter_fourcc(*'XVID'),
                        #  cv2.VideoWriter_fourcc('m', 'p', '4', 'v'),
                         int(cap.get(cv2.CAP_PROP_FPS)), # FPS
                         (852,480)) # frame size

# Loop through the video frames
i=0
while cap.isOpened():
    i+=1
    # Read a frame from the video
    success, frame = cap.read()

    if success:

        # print(frame.shape)
        frame = cv2.resize(frame, dsize=(852,480))# reshape to 480, 852
        # print(frame.shape)

        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True,tracker="bytetrack.yaml") # , show=True, stream=True

        try:
            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 90 tracks for 90 frames
                    track.pop(0)

                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(255,0,0), thickness=10)

        except:
            annotated_frame = frame

        # Display the annotated frame (requires X-forwarding ...)
        # cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # write the frame to the output file
        output_video.write(annotated_frame)

        # save every n-th frame as jpg
        n=50
        if i % n == 0:
            print('saving '+str(n)+'-th frame ...')
            cv2.imwrite("../output/"+fname+"_frame_%d.jpg" % i, annotated_frame)     # save frame as JPEG file 

        # except:
        #     print("No detection in this frame, skipping ...")



        if i==800:
            print("ok, that's enough ...")
            break

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        print("failed to read the frame :<")
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
output_video.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 704.3ms
Speed: 76.4ms preprocess, 704.3ms inference, 19.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 557.5ms
Speed: 10.0ms preprocess, 557.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 185.6ms
Speed: 4.0ms preprocess, 185.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 156.7ms
Speed: 3.0ms preprocess, 156.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 242.1ms
Speed: 3.0ms preprocess, 242.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 312.0ms
Speed: 59.2ms preprocess, 312.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 210.1ms
Speed: 3.0ms preprocess, 210.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 209.0ms
Speed: 4.0ms preprocess